In [ ]:
# read raster

In [39]:
import rasterio
import os
import numpy as np
%matplotlib inline

# Data dir
path = "C:/Users/didar/Downloads/data_1/data/"
fp = os.path.join(path, "bigElk_dem.tif")

# Open the file:
dem = rasterio.open(fp)


In [45]:
"""
Author: Originally created by Galen Maclaurin, updated by Ricardo Oliveira
Created: Created on 3.15.16, updated on 10.17.19
Purpose: Helper functions to get started with Lab 5
"""

import numpy as np


def slopeAspect(dem, cs):
    """Calculates slope and aspect using the 3rd-order finite difference method

    Parameters
    ----------
    dem : numpy array
        A numpy array of a DEM
    cs : float
        The cell size of the original DEM

    Returns
    -------
    numpy arrays
        Slope and Aspect arrays
    """

    from math import pi
    from scipy import ndimage
    kernel = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
    dzdx = ndimage.convolve(dem, kernel, mode='mirror') / (8 * cs)
    dzdy = ndimage.convolve(dem, kernel.T, mode='mirror') / (8 * cs)
    slp = np.arctan((dzdx ** 2 + dzdy ** 2) ** 0.5) * 180 / pi
    ang = np.arctan2(-dzdy, dzdx) * 180 / pi
    aspect = np.where(ang > 90, 450 - ang, 90 - ang)
    return slp, aspect


def reclassAspect(npArray):
    """Reclassify aspect array to 8 cardinal directions (N,NE,E,SE,S,SW,W,NW),
    encoded 1 to 8, respectively (same as ArcGIS aspect classes).

    Parameters
    ----------
    npArray : numpy array
        numpy array with aspect values 0 to 360

    Returns
    -------
    numpy array
        numpy array with cardinal directions
    """
    return np.where((npArray > 22.5) & (npArray <= 67.5), 2,
    np.where((npArray > 67.5) & (npArray <= 112.5), 3,
    np.where((npArray > 112.5) & (npArray <= 157.5), 4,
    np.where((npArray > 157.5) & (npArray <= 202.5), 5,
    np.where((npArray > 202.5) & (npArray <= 247.5), 6,
    np.where((npArray > 247.5) & (npArray <= 292.5), 7,
    np.where((npArray > 292.5) & (npArray <= 337.5), 8, 1)))))))


def reclassByHisto(npArray, bins):
    """Reclassify np array based on a histogram approach using a specified
    number of bins. Returns the reclassified numpy array and the classes from
    the histogram.

    Parameters
    ----------
    npArray : numpy array
        Array to be reclassified
    bins : int
        Number of bins

    Returns
    -------
    numpy array
        umpy array with reclassified values
    """
    # array = np.where(np.isnan(npArray), 0, npArray)
    histo = np.histogram(~np.isnan(npArray), bins)[1]
    rClss = np.zeros_like(npArray)
    for i in range(bins):
        #print(i + 1, histo[i], histo[i + 1])
        #print(np.where((npArray > histo[i]) & (npArray <= histo[i + 1])))
        rClss = np.where((npArray >= histo[i]) & (npArray <= histo[i + 1]),
                         i + 1, rClss)
    return rClss

In [46]:
# dem cell size
cellX, cellY  =dem.res
print(cellX, cellY)

30.0 30.0


In [47]:
def dem_process(img):
    cellX=img.res
    slope, aspect=slopeAspect(img, cellX)
    reclass=reclassAspect(slope)
    

In [48]:
dem_=dem.read(1)
slope, aspect=slopeAspect(dem_,cellX)

In [49]:
aspect_reclass=reclassAspect(aspect)
slope_reclass=reclassByHisto(slope, 10)

In [50]:
import glob
bands=glob.glob(path+'L5_big_elk/*'+'.tif')


In [51]:
b3=[]
b4=[]
for i in bands:
    if i[-6:-4]=='B3':
        b3.append(i)
    else:
        b4.append(i)

In [52]:
def ndvi(red,nri):
    red1 = rasterio.open(red)
    red_=red1.read(1)
    nri = rasterio.open(nri)
    nri_=nri.read(1)
    return (nri_-red_)/(nri_+red_)

In [53]:
ndvi_list=[]
for a,b in zip(b3,b4):
    ndvi_=ndvi(a,b)
    ndvi_list.append(ndvi_)

In [54]:
ndvi_list

[array([[0.15254237, 0.14705883, 0.10810811, ..., 0.28125   , 0.2982456 ,
         0.2542373 ],
        [0.24      , 0.16363636, 0.10769231, ..., 0.29411766, 0.22857143,
         0.24324325],
        [0.27272728, 0.10638298, 0.18181819, ..., 0.2835821 , 0.17142858,
         0.20689656],
        ...,
        [0.34615386, 0.31914893, 0.39130434, ..., 0.32142857, 0.3877551 ,
         0.3478261 ],
        [0.39130434, 0.3617021 , 0.33333334, ..., 0.34615386, 0.39130434,
         0.30232558],
        [0.30612245, 0.26923078, 0.28301886, ..., 0.33333334, 0.31707317,
         0.27659574]], dtype=float32),
 array([[0.41025642, 0.41573033, 0.3043478 , ..., 0.3243243 , 0.30555555,
         0.30769232],
        [0.32307693, 0.3611111 , 0.3493976 , ..., 0.33333334, 0.23255815,
         0.27272728],
        [0.3125    , 0.17460318, 0.2881356 , ..., 0.2857143 , 0.29268292,
         0.3253012 ],
        ...,
        [0.44827586, 0.42857143, 0.4385965 , ..., 0.3529412 , 0.3939394 ,
         0.4       

In [55]:
b3

['C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2002_B3.tif',
 'C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2003_B3.tif',
 'C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2004_B3.tif',
 'C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2005_B3.tif',
 'C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2006_B3.tif',
 'C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2007_B3.tif',
 'C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2008_B3.tif',
 'C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2009_B3.tif',
 'C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2010_B3.tif',
 'C:/Users/didar/Downloads/data_1/data/L5_big_elk\\L5034032_2011_B3.tif']

In [263]:
np.unique(healthy_forest)

array([  1,   2, 255], dtype=uint8)

In [143]:
healthy_forest_=healthy_forest==2

In [144]:
good_forest=healthy_forest_*1
good_forest

array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]])

In [194]:
mean_ndvi=[]
for i in ndvi_list:
    mean_ndvi.append(np.mean(i[good_forest==1]))

In [195]:
mean_ndvi

[0.28746116,
 0.3476631,
 0.31536812,
 0.347109,
 0.36434868,
 0.35422683,
 0.32722387,
 0.3645139,
 0.33309895,
 0.33822393]

In [153]:
# rr calculation for all ndvi
rr=[]
for i,j in zip(ndvi_list,mean_ndvi):
    rr.append(i/j)


In [154]:
# flattening arry
arrr=[]
for i in rr:
    arr=np.ravel(i)
    arrr.append(arr)

In [155]:
np.array(arrr)

array([[0.7316144 , 0.70531464, 0.5185015 , ..., 1.598713  , 1.520727  ,
        1.3265916 ],
       [1.6269257 , 1.6486332 , 1.2069312 , ..., 1.7393119 , 1.6777672 ,
        1.416297  ],
       [1.4124047 , 1.224084  , 0.8530202 , ..., 1.7106764 , 1.651542  ,
        1.5812635 ],
       ...,
       [1.4126691 , 1.5622574 , 1.460495  , ..., 1.4383423 , 1.5009158 ,
        1.3383551 ],
       [1.1906775 , 1.1199707 , 0.9149416 , ..., 1.4732112 , 1.5919315 ,
        1.5803537 ],
       [1.0999552 , 1.320493  , 1.2175974 , ..., 1.6448246 , 1.5678095 ,
        1.5678095 ]], dtype=float32)

In [156]:
import pandas as pd
df=pd.DataFrame(np.array(arrr))
df

,0,1,2,3,4,5,6,7,8,9,...,128510,128511,128512,128513,128514,128515,128516,128517,128518,128519
0,0.731614,0.705315,0.518502,0.504857,0.591305,0.599517,0.773571,0.666130,0.570969,0.560588,...,1.348914,1.547142,1.488457,1.726610,2.055488,1.900357,1.786797,1.598713,1.520727,1.326592
1,1.626926,1.648633,1.206931,0.948303,0.991408,0.930210,1.004627,0.737792,0.699817,0.969377,...,1.081536,1.126044,1.252305,1.525243,1.718440,1.718440,1.699556,1.739312,1.677767,1.416297
2,1.412405,1.224084,0.853020,0.746393,0.830075,0.850058,0.771482,0.874346,0.736808,1.080074,...,1.157222,1.285803,1.277890,1.619159,1.681434,1.781075,1.567223,1.710676,1.651542,1.581264
3,1.387794,1.269390,1.021362,0.953271,0.891665,0.811476,0.812447,0.676079,0.687455,0.794393,...,1.004954,1.065649,1.256950,1.482075,1.649892,1.600642,1.482075,1.602722,1.506606,1.281278
4,1.261341,1.424094,1.386226,1.234786,1.069398,0.946006,0.987136,0.802671,0.873236,1.103673,...,0.991054,1.048584,1.197475,1.332402,1.586848,1.669422,1.480704,1.419008,1.346855,1.300758
5,1.602650,1.793013,1.702815,1.378470,1.184567,1.076552,1.015343,0.904237,0.764529,1.035526,...,0.983730,1.061495,1.144750,1.536375,1.546196,1.504964,1.504964,1.513614,1.488175,1.373700
6,1.440000,1.598161,1.607148,1.404445,1.132617,1.020070,0.972308,0.877778,0.702222,0.940842,...,1.053334,1.077830,1.304127,1.586840,1.564953,1.720939,1.638519,1.697911,1.580001,1.363138
7,1.412669,1.562257,1.460495,1.235803,0.926279,0.756462,0.756462,0.662879,0.589901,0.770470,...,1.008615,1.170714,1.230389,1.357751,1.668665,1.470897,1.502561,1.438342,1.500916,1.338355
8,1.190678,1.119971,0.914942,0.864191,0.673794,0.577538,0.648281,0.523325,0.506819,0.640055,...,1.061288,1.158926,1.340810,1.464577,1.712699,1.681478,1.735719,1.473211,1.591931,1.580354
9,1.099955,1.320493,1.217597,0.980377,0.815261,0.611446,0.567587,0.582329,0.557810,0.671918,...,1.032664,1.211874,1.320493,1.488175,1.577924,1.536967,1.630522,1.644825,1.567809,1.567809


In [157]:
# polyfit
def polyfit_(df,columns):
    coeff_list=[]
    for i in columns:
        data=df[i]
        coeff = np.polyfit(data,data.index,1)
        coeff_list.append(coeff[0])
    return coeff_list


In [158]:
coeff=polyfit_(df,columns)

In [159]:
coeff

[0.6085769091730946,
 2.753284857953963,
 3.8412326600845152,
 5.128283399681182,
 1.6823405258935131,
 -3.5773926075405567,
 -8.7405911989017,
 -8.440598794083389,
 -10.093904547376226,
 -3.618190974767285,
 -0.453875211435455,
 3.074759792965806,
 2.6088091138152243,
 7.4781671191793295,
 2.9103326820337005,
 -1.9681227681499283,
 0.9382817865032952,
 2.0828271835578414,
 2.6393978970910017,
 1.3500793055075704,
 -1.080604284517025,
 -0.974631800252657,
 1.581333710389401,
 -1.322791723659587,
 1.5729591359728385,
 5.013586074704265,
 7.349608753202538,
 1.6116864259300412,
 -5.366780164611886,
 0.3896499709675212,
 -11.807555389406343,
 -4.415902012376381,
 -13.598086017495069,
 -16.141308931790938,
 -15.039044776291169,
 -15.26622792649589,
 -13.252818083149785,
 -0.09940523905421449,
 15.375054050778624,
 1.5524151140538218,
 2.7634831814610052,
 3.9142676560827625,
 4.2127259905706715,
 -12.865796161637578,
 -6.868893941963225,
 5.825754839264096,
 -11.784346150033684,
 8.0253818

In [160]:
np.shape(dem)

(280, 459)

In [161]:
reshape_coeff=np.reshape(coeff, (280,459))

In [162]:
# print mean coef  only for healthy forest
np.mean(reshape_coeff[good_forest==1])

-0.5319704320869123

In [166]:
# print mean rr for healty forest
for i in rr:
    area_mask=i[good_forest==1]
    print(np.mean(area_mask))

0.99999994
0.9999998
0.9999998
1.0
1.0000001
0.99999994
1.0
1.0
1.0
1.0


In [206]:
# defining funciton for zonal statistics
def min(coef,cls):
    min_=[]
    cls=cls[good_forest==1]
    coef=coef[good_forest==1]
    for i in np.unique(cls):
        mask=coef[cls==i]
        min_.append(np.min(mask))
    return min_

def max(coef,cls):
    max_=[]
    cls=cls[good_forest==1]
    coef=coef[good_forest==1]
    for i in np.unique(cls):
        mask=coef[cls==i]
        max_.append(np.max(mask))
    return max_
               
def mean(coef,cls):
    mean_=[]
    cls=cls[good_forest==1]
    coef=coef[good_forest==1]
    for i in np.unique(cls):
        mask=coef[cls==i]
        mean_.append(np.mean(mask))
    return mean_
               
def std(coef,cls):
    std_=[]
    cls=cls[good_forest==1]
    coef=coef[good_forest==1]
    for i in np.unique(cls):
        mask=coef[cls==i]
        std_.append(np.std(mask))
    return std_    

def count(coef,cls):
    count_=[]
    cls=cls[good_forest==1]
    coef=coef[good_forest==1]
    for i in np.unique(cls):
        mask=coef[cls==i]
        count_.append(len(np.ravel(mask)))
    return count_ 

In [207]:
mean(reshape_coeff,aspect_reclass)

[1.2636295897735172,
 -1.7203564088677825,
 -3.1197137347329535,
 -1.853055267891145,
 0.07813632370359838,
 1.0131603517155032,
 2.2456408868061173,
 3.2028815876428447]

In [208]:
for i in np.unique(aspect_reclass):
    print(i)

1
2
3
4
5
6
7
8


In [209]:
aspect_ = pd.DataFrame(list(zip(np.unique(aspect_reclass), min(reshape_coeff,aspect_reclass),max(reshape_coeff,aspect_reclass),mean(reshape_coeff,aspect_reclass),
                          std(reshape_coeff,aspect_reclass), count(reshape_coeff,aspect_reclass))),
               columns =['aspect_class', 'min','max','mean','std','count'])
aspect_

,aspect_class,min,max,mean,std,count
0,1,-52.688603,48.444985,1.263630,10.802869,10585
1,2,-57.436036,50.608208,-1.720356,11.208195,15109
2,3,-63.830691,55.315029,-3.119714,11.161124,20125
3,4,-64.063271,52.569008,-1.853055,12.203112,12729
4,5,-77.542645,83.448832,0.078136,12.696095,10527
5,6,-47.393166,55.432824,1.013160,12.715849,9832
6,7,-46.707394,57.597616,2.245641,11.877223,7543
7,8,-43.818233,50.239392,3.202882,10.907680,6768


In [210]:
slope_ = pd.DataFrame(list(zip(np.unique(slope_reclass), min(reshape_coeff,slope_reclass),max(reshape_coeff,slope_reclass),mean(reshape_coeff,slope_reclass),
                          std(reshape_coeff,slope_reclass), count(reshape_coeff,slope_reclass))),
               columns =['slope_reclass', 'min','max','mean','std','count'])
slope_

,slope_reclass,min,max,mean,std,count
0,0.0,-77.542645,83.448832,-0.539623,11.863513,92582
1,1.0,-35.739980,42.131186,-0.425320,11.382490,42
2,2.0,-19.737040,41.722880,0.778993,10.894539,38
3,3.0,-20.188487,19.561486,0.102969,7.819933,51
4,4.0,-22.341363,19.914696,0.604448,8.199224,64
5,5.0,-18.094502,23.885949,1.050080,8.922451,73
6,6.0,-21.015946,31.532013,0.197595,8.857376,51
7,7.0,-28.436143,20.660975,0.175770,8.932877,75
8,8.0,-25.002149,14.538653,-0.247211,8.685283,67
9,9.0,-29.959814,20.722700,1.632653,8.922474,96


In [245]:
mask_coeff=reshape_coeff*healthy_forest_

In [257]:
mask_coeff

array([[  0.60857691,   2.75328486,   3.84123266, ...,  -7.18521832,
        -20.32753203, -16.27822623],
       [-20.71522036,  -2.52687515,  -1.0028649 , ..., -15.33474854,
         19.39482355, -12.05203223],
       [ -8.62261811,  11.47634066,  -6.62383202, ...,  -4.7763111 ,
         10.20332756,   5.28987602],
       ...,
       [ -7.70731961,  11.13818535, -17.07605403, ...,  -9.22982017,
        -14.56881006, -14.77922275],
       [ -5.1316822 , -14.03078201, -11.56248285, ..., -15.15741268,
        -14.61975671,  14.95589223],
       [  8.96952463, -16.18484241, -36.86744401, ...,  -9.86857154,
         -3.58183052,   9.51711078]])

In [258]:

mask_coeff[mask_coeff == 0.] = -99
mask_coeff

array([[  0.60857691,   2.75328486,   3.84123266, ...,  -7.18521832,
        -20.32753203, -16.27822623],
       [-20.71522036,  -2.52687515,  -1.0028649 , ..., -15.33474854,
         19.39482355, -12.05203223],
       [ -8.62261811,  11.47634066,  -6.62383202, ...,  -4.7763111 ,
         10.20332756,   5.28987602],
       ...,
       [ -7.70731961,  11.13818535, -17.07605403, ...,  -9.22982017,
        -14.56881006, -14.77922275],
       [ -5.1316822 , -14.03078201, -11.56248285, ..., -15.15741268,
        -14.61975671,  14.95589223],
       [  8.96952463, -16.18484241, -36.86744401, ...,  -9.86857154,
         -3.58183052,   9.51711078]])

In [260]:
np.shape(mask_coeff)

(280, 459)

In [262]:
with rasterio.open(path+'bigElk_dem.tif') as src:
    ras_meta = src.profile

with rasterio.open(path+'coeff.tif', 'w', **ras_meta) as dst:
    dst.write(mask_coeff, indexes=1)